In [1]:
from pathlib import Path

from matflow import load_workflow
# from plotly.graph_objects import FigureWidget
import matplotlib.pyplot as plt
import matplotlib as mpl

%matplotlib widget

## Load workflow

In [2]:
workflow_path = Path('/home/mike/Desktop/csf-home/scratch/RVE_extrusion_2021-01-28-130944/')
workflow = load_workflow(workflow_path)

Loading matflow config from /home/mike/.matflow/config.yml
Loading task schemas from 1 file(s)...OK!
Loading software definitions from 1 file(s)...OK!


/opt/anaconda3/lib/python3.7/site-packages/matflow/models/construction.py:491: UserWarning: Input parameter "homog_label" for task "generate_volume_element" with task context "" has both a local value and a non-local value. The non-local value is derived from an input of task "modify_volume_element" with task context "". The local value will be used.
  warn(msg)
/opt/anaconda3/lib/python3.7/site-packages/matflow/models/construction.py:491: UserWarning: Input parameter "homog_label" for task "modify_volume_element" with task context "" has both a local value and a non-local value. The non-local value is derived from an input of task "generate_volume_element" with task context "". The local value will be used.
  warn(msg)


In [3]:
workflow.tasks

[Task(name='load_microstructure', method='EBSD+DIC', software='defdap'),
 Task(name='generate_volume_element', method='extrusion', software='damask'),
 Task(name='modify_volume_element', method='add_buffer_zones', software='damask'),
 Task(name='visualise_volume_element', method='VTK', software='damask'),
 Task(name='generate_load_case', method='uniaxial', software='formable'),
 Task(name='simulate_volume_element_loading', method='CP_FFT', software='damask')]

In [4]:
vol_elem_resp = workflow.tasks[5].elements[0].outputs.volume_element_response

In [5]:
mean_strain = vol_elem_resp['incremental_data']['vol_avg_strain']['data']
mean_stress = vol_elem_resp['incremental_data']['vol_avg_stress']['data']

In [6]:
plt.figure()
plt.plot(mean_strain[:, 0, 0], mean_stress[:, 0, 0])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [7]:
vol_elem_resp['field_data']['increments']

[10, 20, 30]

In [8]:
stress = vol_elem_resp['field_data']['sigma']
strain = vol_elem_resp['field_data']['epsilon_U^2(F)']
print(stress.shape)
print(strain.shape)

(3, 3, 3, 280, 240, 12)
(3, 3, 3, 280, 240, 12)


In [11]:
plt.figure()
plt.imshow(stress[2, 0, 0, :, :, 5].T/1e6)
plt.colorbar(label='Stress (MPa)')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [12]:
plt.figure()
plt.imshow(strain[2, 0, 0, :, :, 5].T)
plt.colorbar(label='Strain')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …